In [29]:
import os.path
import shutil

import feature_engine.encoding
import feature_engine.imputation
import feature_engine.pipeline
import feature_engine.selection
import feature_engine.wrappers
import pandas
import sklearn.model_selection
import sklearn.preprocessing
import tensorflow.keras.callbacks
import tensorflow.keras.layers
import tensorflow.keras.models

In [3]:
train_df = pandas.read_csv("train.csv")
train_df

,id,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,...,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data,outcome
0,0,yes,adult,530001,38.1,132.0,24.0,cool,reduced,dark_cyanotic,...,57.0,8.5,serosanguious,3.4,yes,2209,0,0,no,died
1,1,yes,adult,533836,37.5,88.0,12.0,cool,normal,pale_cyanotic,...,33.0,64.0,serosanguious,2.0,yes,2208,0,0,no,euthanized
2,2,yes,adult,529812,38.3,120.0,28.0,cool,reduced,pale_pink,...,37.0,6.4,serosanguious,3.4,yes,5124,0,0,no,lived
3,3,yes,adult,5262541,37.1,72.0,30.0,cold,reduced,pale_pink,...,53.0,7.0,cloudy,3.9,yes,2208,0,0,yes,lived
4,4,no,adult,5299629,38.0,52.0,48.0,normal,normal,normal_pink,...,47.0,7.3,cloudy,2.6,no,0,0,0,yes,lived
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1230,1230,yes,adult,535246,38.5,129.0,48.0,cool,reduced,pale_pink,...,57.0,66.0,serosanguious,2.0,yes,2206,0,0,no,lived
1231,1231,yes,adult,528570,37.5,60.0,50.0,cool,reduced,pale_cyanotic,...,35.0,6.4,serosanguious,3.6,yes,2209,0,0,yes,died
1232,1232,yes,young,529685,37.5,84.0,40.0,normal,reduced,normal_pink,...,40.0,5.9,cloudy,7.0,yes,400,0,0,yes,lived
1233,1233,yes,adult,534784,38.1,70.0,16.0,normal,reduced,bright_red,...,58.0,74.0,cloudy,2.0,yes,2209,0,0,no,lived


In [5]:
train_wo_na_df = train_df.dropna()
train_wo_na_df

,id,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,...,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data,outcome
0,0,yes,adult,530001,38.1,132.0,24.0,cool,reduced,dark_cyanotic,...,57.0,8.5,serosanguious,3.4,yes,2209,0,0,no,died
1,1,yes,adult,533836,37.5,88.0,12.0,cool,normal,pale_cyanotic,...,33.0,64.0,serosanguious,2.0,yes,2208,0,0,no,euthanized
3,3,yes,adult,5262541,37.1,72.0,30.0,cold,reduced,pale_pink,...,53.0,7.0,cloudy,3.9,yes,2208,0,0,yes,lived
4,4,no,adult,5299629,38.0,52.0,48.0,normal,normal,normal_pink,...,47.0,7.3,cloudy,2.6,no,0,0,0,yes,lived
5,5,no,adult,529642,38.1,56.0,32.0,normal,normal,bright_pink,...,49.0,8.0,cloudy,2.8,no,0,0,0,yes,lived
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1228,1228,yes,adult,534963,38.3,48.0,20.0,normal,normal,bright_pink,...,41.0,65.0,clear,2.0,yes,3111,0,0,no,lived
1229,1229,yes,adult,535338,38.0,136.0,20.0,cool,reduced,bright_red,...,66.0,65.0,serosanguious,2.0,yes,2208,0,0,no,lived
1231,1231,yes,adult,528570,37.5,60.0,50.0,cool,reduced,pale_cyanotic,...,35.0,6.4,serosanguious,3.6,yes,2209,0,0,yes,died
1232,1232,yes,young,529685,37.5,84.0,40.0,normal,reduced,normal_pink,...,40.0,5.9,cloudy,7.0,yes,400,0,0,yes,lived


In [6]:
X_train_val = train_wo_na_df.drop(columns=["outcome"])
X_train_val

,id,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,...,abdomen,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data
0,0,yes,adult,530001,38.1,132.0,24.0,cool,reduced,dark_cyanotic,...,distend_small,57.0,8.5,serosanguious,3.4,yes,2209,0,0,no
1,1,yes,adult,533836,37.5,88.0,12.0,cool,normal,pale_cyanotic,...,distend_small,33.0,64.0,serosanguious,2.0,yes,2208,0,0,no
3,3,yes,adult,5262541,37.1,72.0,30.0,cold,reduced,pale_pink,...,distend_small,53.0,7.0,cloudy,3.9,yes,2208,0,0,yes
4,4,no,adult,5299629,38.0,52.0,48.0,normal,normal,normal_pink,...,normal,47.0,7.3,cloudy,2.6,no,0,0,0,yes
5,5,no,adult,529642,38.1,56.0,32.0,normal,normal,bright_pink,...,firm,49.0,8.0,cloudy,2.8,no,0,0,0,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1228,1228,yes,adult,534963,38.3,48.0,20.0,normal,normal,bright_pink,...,distend_small,41.0,65.0,clear,2.0,yes,3111,0,0,no
1229,1229,yes,adult,535338,38.0,136.0,20.0,cool,reduced,bright_red,...,distend_small,66.0,65.0,serosanguious,2.0,yes,2208,0,0,no
1231,1231,yes,adult,528570,37.5,60.0,50.0,cool,reduced,pale_cyanotic,...,distend_small,35.0,6.4,serosanguious,3.6,yes,2209,0,0,yes
1232,1232,yes,young,529685,37.5,84.0,40.0,normal,reduced,normal_pink,...,firm,40.0,5.9,cloudy,7.0,yes,400,0,0,yes


In [7]:
y_train_val = train_wo_na_df[["outcome"]]
y_train_val

,outcome
0,died
1,euthanized
3,lived
4,lived
5,lived
...,...
1228,lived
1229,lived
1231,died
1232,lived


In [8]:
columns = list(X_train_val.columns)
columns

['id',
 'surgery',
 'age',
 'hospital_number',
 'rectal_temp',
 'pulse',
 'respiratory_rate',
 'temp_of_extremities',
 'peripheral_pulse',
 'mucous_membrane',
 'capillary_refill_time',
 'pain',
 'peristalsis',
 'abdominal_distention',
 'nasogastric_tube',
 'nasogastric_reflux',
 'nasogastric_reflux_ph',
 'rectal_exam_feces',
 'abdomen',
 'packed_cell_volume',
 'total_protein',
 'abdomo_appearance',
 'abdomo_protein',
 'surgical_lesion',
 'lesion_1',
 'lesion_2',
 'lesion_3',
 'cp_data']

In [9]:
X_train_val.dtypes

id                         int64
surgery                   object
age                       object
hospital_number            int64
rectal_temp              float64
pulse                    float64
respiratory_rate         float64
temp_of_extremities       object
peripheral_pulse          object
mucous_membrane           object
capillary_refill_time     object
pain                      object
peristalsis               object
abdominal_distention      object
nasogastric_tube          object
nasogastric_reflux        object
nasogastric_reflux_ph    float64
rectal_exam_feces         object
abdomen                   object
packed_cell_volume       float64
total_protein            float64
abdomo_appearance         object
abdomo_protein           float64
surgical_lesion           object
lesion_1                   int64
lesion_2                   int64
lesion_3                   int64
cp_data                   object
dtype: object

In [10]:
numerical_columns = X_train_val.select_dtypes(include=["int64", "float64"]).columns
numerical_columns = list(numerical_columns)
numerical_columns.remove("id")
numerical_columns

['hospital_number',
 'rectal_temp',
 'pulse',
 'respiratory_rate',
 'nasogastric_reflux_ph',
 'packed_cell_volume',
 'total_protein',
 'abdomo_protein',
 'lesion_1',
 'lesion_2',
 'lesion_3']

In [11]:
categorical_columns = X_train_val.select_dtypes(include=["object"]).columns
categorical_columns = list(categorical_columns)
categorical_columns

['surgery',
 'age',
 'temp_of_extremities',
 'peripheral_pulse',
 'mucous_membrane',
 'capillary_refill_time',
 'pain',
 'peristalsis',
 'abdominal_distention',
 'nasogastric_tube',
 'nasogastric_reflux',
 'rectal_exam_feces',
 'abdomen',
 'abdomo_appearance',
 'surgical_lesion',
 'cp_data']

In [13]:
X_pipeline = feature_engine.pipeline.make_pipeline(
    feature_engine.selection.DropFeatures(features_to_drop=["id"]),
    feature_engine.encoding.OrdinalEncoder(encoding_method="arbitrary", variables=categorical_columns, missing_values="ignore"),
    feature_engine.wrappers.SklearnTransformerWrapper(transformer=sklearn.preprocessing.MinMaxScaler(), variables=numerical_columns),
)  

In [14]:
X_train_val_transformed = X_pipeline.fit_transform(X_train_val)
X_train_val_transformed

,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,...,abdomen,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data
0,0,0,0.001424,0.500000,0.662338,0.181818,0,0,0,0,...,0,0.653846,0.058480,0,0.263736,0,0.071006,0.0,0.0,0
1,0,0,0.002226,0.388889,0.376623,0.045455,0,1,1,0,...,0,0.192308,0.707602,0,0.109890,0,0.070974,0.0,0.0,0
3,0,0,0.991094,0.314815,0.272727,0.250000,1,0,2,0,...,0,0.576923,0.040936,1,0.318681,0,0.070974,0.0,0.0,1
4,1,0,0.998850,0.481481,0.142857,0.454545,2,1,3,1,...,1,0.461538,0.044444,1,0.175824,1,0.000000,0.0,0.0,1
5,1,0,0.001349,0.500000,0.168831,0.272727,2,1,4,1,...,2,0.500000,0.052632,1,0.197802,1,0.000000,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1228,0,0,0.002462,0.537037,0.116883,0.136364,2,1,4,0,...,0,0.346154,0.719298,2,0.109890,0,0.100000,0.0,0.0,0
1229,0,0,0.002540,0.481481,0.688312,0.136364,0,0,5,1,...,0,0.826923,0.719298,0,0.109890,0,0.070974,0.0,0.0,0
1231,0,0,0.001125,0.388889,0.194805,0.477273,0,0,1,1,...,0,0.230769,0.033918,0,0.285714,0,0.071006,0.0,0.0,1
1232,0,1,0.001358,0.388889,0.350649,0.363636,2,0,3,1,...,2,0.326923,0.028070,1,0.659341,0,0.012858,0.0,0.0,1


In [15]:
y_encoder = feature_engine.encoding.OrdinalEncoder(encoding_method="arbitrary")

In [16]:
y_train_val_encoded = y_encoder.fit_transform(y_train_val)
y_train_val_encoded

,outcome
0,0
1,1
3,2
4,2
5,2
...,...
1228,2
1229,2
1231,0
1232,2


In [31]:
if os.path.isdir("logs"):
    shutil.rmtree("logs")

model = tensorflow.keras.models.Sequential()
model.add(tensorflow.keras.layers.Dense(64, activation="relu"))
model.add(tensorflow.keras.layers.Dense(3, activation="softmax"))
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
callbacks = [
    tensorflow.keras.callbacks.EarlyStopping(),
    tensorflow.keras.callbacks.TensorBoard(),
]
model.fit(X_train_val_transformed, y_train_val_encoded, callbacks=callbacks, epochs=20, validation_split=0.2)

Epoch 1/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.3080 - loss: 1.3570 - val_accuracy: 0.4387 - val_loss: 1.0141
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4718 - loss: 0.9999 - val_accuracy: 0.4903 - val_loss: 0.9516
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5479 - loss: 0.9274 - val_accuracy: 0.5419 - val_loss: 0.9118
Epoch 4/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6364 - loss: 0.8551 - val_accuracy: 0.5871 - val_loss: 0.8808
Epoch 5/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6423 - loss: 0.8053 - val_accuracy: 0.5935 - val_loss: 0.8618
Epoch 6/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6631 - loss: 0.7970 - val_accuracy: 0.6323 - val_loss: 0.8506
Epoch 7/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6593 - loss: 0.7743 - val_accuracy: 0.6645 - val_loss: 0.8355
Epoch 8/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6941 - loss: 0.7167 - val_accuracy: 0.6710 - val_loss

In [32]:
%load_ext tensorboard
%tensorboard --logdir logs

In [33]:
test_df = pandas.read_csv("test.csv")
test_df

,id,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,...,abdomen,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data
0,1235,no,adult,534053,38.6,40.0,20.0,normal,normal,normal_pink,...,distend_small,42.0,7.5,clear,2.3,no,0,0,0,no
1,1236,yes,adult,528469,38.2,112.0,48.0,cool,reduced,bright_pink,...,distend_small,44.0,6.0,serosanguious,2.6,no,2208,0,0,yes
2,1237,yes,adult,528178,37.7,66.0,12.0,cool,normal,bright_red,...,distend_small,31.5,6.0,cloudy,1.6,yes,2205,0,0,yes
3,1238,no,adult,534784,37.1,88.0,20.0,cool,reduced,pale_cyanotic,...,distend_large,75.0,81.0,NaN,1.0,yes,1400,0,0,no
4,1239,yes,adult,529840,38.3,50.0,12.0,NaN,normal,bright_pink,...,distend_small,37.0,6.8,cloudy,2.6,yes,2208,0,0,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
819,2054,no,adult,529461,40.3,114.0,36.0,cool,reduced,normal_pink,...,distend_large,57.0,8.1,serosanguious,4.5,yes,3205,0,0,yes
820,2055,yes,adult,535338,37.2,100.0,20.0,cool,reduced,pale_cyanotic,...,distend_small,50.0,66.0,serosanguious,2.0,yes,2209,0,0,no
821,2056,yes,adult,529640,39.2,132.0,12.0,cool,reduced,dark_cyanotic,...,NaN,53.0,7.6,serosanguious,4.5,yes,2205,0,0,no
822,2057,no,adult,5287179,38.3,54.0,66.0,normal,normal,normal_pink,...,NaN,49.0,8.6,clear,5.0,no,3111,0,0,yes


In [34]:
tmp_df = test_df.copy()
test_transformed_df = X_pipeline.transform(tmp_df)
test_transformed_df

C:\Users\hsmtk\repo\musical-sniffle\.venv\Lib\site-packages\feature_engine\encoding\base_encoder.py:260: UserWarning: During the encoding, NaN values were introduced in the feature(s) temp_of_extremities, peripheral_pulse, mucous_membrane, capillary_refill_time, pain, peristalsis, abdominal_distention, nasogastric_tube, nasogastric_reflux, rectal_exam_feces, abdomen, abdomo_appearance.
  warnings.warn(


,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,...,abdomen,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data
0,1,0,0.002272,0.592593,0.064935,0.136364,2.0,1.0,3.0,1.0,...,0.0,0.365385,0.046784,2.0,0.142857,1,0.000000,0.0,0.0,0
1,0,0,0.001104,0.518519,0.532468,0.454545,0.0,0.0,4.0,0.0,...,0.0,0.403846,0.029240,0.0,0.175824,1,0.070974,0.0,0.0,1
2,0,0,0.001043,0.425926,0.233766,0.045455,0.0,1.0,5.0,1.0,...,0.0,0.163462,0.029240,1.0,0.065934,0,0.070878,0.0,0.0,1
3,1,0,0.002425,0.314815,0.376623,0.136364,0.0,0.0,1.0,1.0,...,3.0,1.000000,0.906433,NaN,0.000000,0,0.045002,0.0,0.0,0
4,0,0,0.001391,0.537037,0.129870,0.045455,NaN,1.0,4.0,1.0,...,0.0,0.269231,0.038596,1.0,0.175824,0,0.070974,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
819,1,0,0.001311,0.907407,0.545455,0.318182,0.0,0.0,3.0,0.0,...,3.0,0.653846,0.053801,0.0,0.384615,0,0.103022,0.0,0.0,1
820,0,0,0.002540,0.333333,0.454545,0.136364,0.0,0.0,1.0,0.0,...,0.0,0.519231,0.730994,0.0,0.109890,0,0.071006,0.0,0.0,0
821,0,0,0.001349,0.703704,0.662338,0.045455,0.0,0.0,0.0,0.0,...,NaN,0.576923,0.047953,0.0,0.384615,0,0.070878,0.0,0.0,0
822,1,0,0.996246,0.537037,0.155844,0.659091,2.0,1.0,3.0,1.0,...,NaN,0.500000,0.059649,2.0,0.439560,1,0.100000,0.0,0.0,1


In [39]:
predicted = model.predict(test_transformed_df).argmax(axis=1)
predicted = pandas.DataFrame(predicted)
predicted.columns = ["outcome"]
predicted

26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


,outcome
0,2
1,0
2,2
3,2
4,2
...,...
819,2
820,0
821,2
822,2


In [40]:
predicted_inversed = y_encoder.inverse_transform(predicted)
predicted_inversed

,outcome
0,lived
1,died
2,lived
3,lived
4,lived
...,...
819,lived
820,died
821,lived
822,lived
